# Context data for the sea level data explorer

In [1]:
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
from datetime import datetime

#get data from worldbank
from pandas_datareader import wb

import glob
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import json
import matplotlib.pyplot as plt

InteractiveShell.ast_node_interactivity = "all"
pd.options.display.max_rows = 200
#matplotlib.rcParams['svg.fonttype'] = 'none'


# Sea Level Data, Stations

In [47]:
stations = pd.read_csv('context_data/sealevel_viz_psmsl_1985_2015.csv', usecols=[2, 4, 5, 6, 7, 9])

stations = stations.drop_duplicates()

stations.rename(columns={'ID': 'id'}, inplace=True)

stations['id'].nunique()
stations.head()

513

,id,location,latitude,longitude,country,continent
0,1,Brest,48.383,-4.495,FRA,europe
31,61,Marseille,43.279,5.354,FRA,europe
62,453,Le Havre,49.482,0.106,FRA,europe
93,467,Cherbourg,49.651,-1.636,FRA,europe
124,468,Dunkerque,51.048,2.367,FRA,europe


In [48]:
tide_data = pd.read_csv('data/dataviz/sealevel_viz_whole_timeseries.csv')

In [49]:
tide_data

,id,1807,1808,1809,1810,1811,1812,1813,1814,1815,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,1,0.0,-51.0,-39.0,-35.0,-27.0,-26.0,-15.0,-17.0,-21.0,...,133.0,135.0,140.0,157.0,158.0,162.0,164.0,176.0,169.0,NaN
1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,263.0,261.0,269.0,273.0,278.0,284.0,283.0,NaN,234.0,NaN
2,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,117.0,116.0,119.0,131.0,133.0,150.0,154.0,164.0,160.0,NaN
3,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,324.0,335.0,323.0,318.0,NaN,NaN,NaN,NaN,NaN,NaN
4,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,130.0,141.0,141.0,148.0,158.0,152.0,149.0,144.0,144.0,NaN
5,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,264.0,273.0,270.0,274.0,276.0,268.0,263.0,269.0,275.0,NaN
6,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,133.0,129.0,128.0,125.0,116.0,127.0,128.0,141.0,157.0,165.0
7,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,201.0,209.0,205.0,208.0,213.0,201.0,193.0,184.0,185.0,NaN
8,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,399.0,407.0,421.0,430.0,442.0,461.0,461.0,460.0,445.0,435.0
9,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-261.0,-237.0,-257.0,-284.0,-271.0,-286.0,-315.0,-326.0,-285.0,NaN


In [50]:
stations.describe()

,id,latitude,longitude
count,513.000000,513.000000,513.000000
mean,825.904483,33.845446,25.302012
std,546.822134,27.489068,97.354383
min,1.000000,-65.246000,-177.360000
25%,368.000000,29.135000,-68.517000
50%,808.000000,38.317000,16.548000
75%,1253.000000,52.167000,129.292000
max,2316.000000,79.550000,178.423000


# Worldbank Data, Emissions

In [51]:
#get the data from worldbank, Stand: 7. April 2017

emissions = pd.read_csv('context_data/c02_emissions_worlbank.csv', usecols=[0, 1, 57])

#emissions['c02_rank'] = emissions['2013'].rank(ascending=False)
emissions.rename(columns={'Country Code': 'country_code'}, inplace=True)
emissions.rename(columns={'Country Name': 'country_name'}, inplace=True)
emissions.rename(columns={'2013': 'co2_emissions'}, inplace=True)

#emissions.sort_values(by='c02_rank').head()

emissions.head()

,country_name,country_code,co2_emissions
0,Aruba,ABW,8.515395
1,Afghanistan,AFG,0.693183
2,Angola,AGO,1.384496
3,Albania,ALB,1.662185
4,Andorra,AND,6.473848


# Sea Level Population Data, National Population Data

In [52]:
sealevel_pop = pd.read_csv('context_data/lecz-urban-rural-population-land-area-estimates_country-90m.csv')

In [53]:
sealevel_pop['ElevationZone'] = sealevel_pop['ElevationZone'].str.strip()
sealevel_pop['ISO3v10'] = sealevel_pop['ISO3v10'].str.strip()
pop = sealevel_pop.set_index(['ElevationZone'])
pop = pop.loc[['Elevations Less Than or Equal To 10m']]

pop = pop.reset_index()

pop_low_elevation = pop.groupby(by=['ISO3v10'])['Population2010'].sum()
pop_low_elevation = pop_low_elevation.to_frame(name = 'coastal_population2010_sum')
pop_low_elevation = pop_low_elevation.reset_index()

pop_low_elevation.head()

,ISO3v10,coastal_population2010_sum
0,ABW,21327
1,AGO,651360
2,AIA,3848
3,ALB,354132
4,ANT,46295


In [54]:
pop_total = sealevel_pop.set_index(['ElevationZone'])
pop_total = pop_total.loc[['Total National Population']]

pop_total = pop_total.reset_index()

pop_total = pop_total.groupby(by=['ISO3v10'])['Population2010'].sum()
pop_total = pop_total.to_frame(name = 'total_population2010_sum')
pop_total = pop_total.reset_index()

pop_total.head()

,ISO3v10,total_population2010_sum
0,ABW,108141
1,AGO,19618432
2,AIA,15645
3,ALB,3215988
4,ANT,202847


# Sea Level PSMSL Trend Data

In [55]:
trend_1985_2015 = pd.read_csv('context_data/trends_1985_2015.csv', usecols=[4,7])
trend_1985_2015.rename(columns={'trend': 'trend_1985_2015'}, inplace=True)
trend_1985_2015.head()

,trend_1985_2015,_id
0,2.385,1
1,3.531,7
2,1.476,8
3,1.484,9
4,2.190,10


In [10]:
pd.qcut(trend_1985_2015['trend_1985_2015'], 5)

0        (2.158, 3.131]
1        (3.131, 4.206]
2        (0.555, 2.158]
3        (0.555, 2.158]
4        (2.158, 3.131]
5        (0.555, 2.158]
6        (4.206, 9.444]
7        (2.158, 3.131]
8      [-20.451, 0.555]
9        (2.158, 3.131]
10       (2.158, 3.131]
11       (0.555, 2.158]
12       (2.158, 3.131]
13       (2.158, 3.131]
14       (2.158, 3.131]
15     [-20.451, 0.555]
16       (4.206, 9.444]
17     [-20.451, 0.555]
18       (0.555, 2.158]
19     [-20.451, 0.555]
20       (0.555, 2.158]
21       (3.131, 4.206]
22     [-20.451, 0.555]
23     [-20.451, 0.555]
24     [-20.451, 0.555]
25     [-20.451, 0.555]
26       (2.158, 3.131]
27     [-20.451, 0.555]
28     [-20.451, 0.555]
29       (2.158, 3.131]
30       (0.555, 2.158]
31       (0.555, 2.158]
32     [-20.451, 0.555]
33     [-20.451, 0.555]
34     [-20.451, 0.555]
35       (2.158, 3.131]
36       (0.555, 2.158]
37       (4.206, 9.444]
38       (0.555, 2.158]
39       (0.555, 2.158]
40       (3.131, 4.206]
41       (2.158,

In [56]:
trend_longest = pd.read_csv('context_data/trends_longest.csv', usecols=[4, 6, 12])
trend_longest.rename(columns={'trend': 'trend_longest'}, inplace=True)
trend_longest.rename(columns={'startYear': 'start_year'}, inplace=True)
trend_longest.head()

,start_year,trend_longest,_id
0,1807,0.97,1
1,1811,0.80,2
2,1968,1.73,3
3,1843,2.12,7
4,1849,1.42,8


# Risk Level Data

In [131]:
riskindex = pd.read_csv('context_data/riskindex.csv', usecols=[0,1,2,3])
riskindex.rename(columns={'Rank': 'risk_rank'}, inplace=True)
riskindex.head()

,risk_rank,Country,risk_index,risk_description
0,1,Vanuatu,36.28,very high
1,2,Tonga,29.33,very high
2,3,Philippines,26.70,very high
3,4,Guatemala,19.88,very high
4,5,Bangladesh,19.17,very high


# Create Sea Level Context Data

In [57]:
#Add emissions
sealevel_context = stations.merge(emissions, left_on='country', right_on='country_code', how='left')

sealevel_context.describe()

,id,latitude,longitude,co2_emissions
count,513.000000,513.000000,513.000000,500.000000
mean,825.904483,33.845446,25.302012,10.213933
std,546.822134,27.489068,97.354383,4.530086
min,1.000000,-65.246000,-177.360000,0.214119
25%,368.000000,29.135000,-68.517000,6.779624
50%,808.000000,38.317000,16.548000,9.764393
75%,1253.000000,52.167000,129.292000,13.532295
max,2316.000000,79.550000,178.423000,16.389758


In [58]:
#Add sealevel population
sealevel_context = sealevel_context.merge(pop_low_elevation, left_on='country', right_on='ISO3v10', how='left')

sealevel_context['id'].nunique()

513

In [59]:
sealevel_context.head()

,id,location,latitude,longitude,country,continent,country_name,country_code,co2_emissions,ISO3v10,coastal_population2010_sum
0,1,Brest,48.383,-4.495,FRA,europe,France,FRA,5.050483,FRA,2983590.0
1,61,Marseille,43.279,5.354,FRA,europe,France,FRA,5.050483,FRA,2983590.0
2,453,Le Havre,49.482,0.106,FRA,europe,France,FRA,5.050483,FRA,2983590.0
3,467,Cherbourg,49.651,-1.636,FRA,europe,France,FRA,5.050483,FRA,2983590.0
4,468,Dunkerque,51.048,2.367,FRA,europe,France,FRA,5.050483,FRA,2983590.0


In [60]:
# clean up data (remove country code)

sealevel_context = sealevel_context.drop('ISO3v10', 1)

In [61]:
#Add national population
sealevel_context = sealevel_context.merge(pop_total, left_on='country', right_on='ISO3v10', how='left')

In [62]:
sealevel_context.head()

,id,location,latitude,longitude,country,continent,country_name,country_code,co2_emissions,coastal_population2010_sum,ISO3v10,total_population2010_sum
0,1,Brest,48.383,-4.495,FRA,europe,France,FRA,5.050483,2983590.0,FRA,63125894.0
1,61,Marseille,43.279,5.354,FRA,europe,France,FRA,5.050483,2983590.0,FRA,63125894.0
2,453,Le Havre,49.482,0.106,FRA,europe,France,FRA,5.050483,2983590.0,FRA,63125894.0
3,467,Cherbourg,49.651,-1.636,FRA,europe,France,FRA,5.050483,2983590.0,FRA,63125894.0
4,468,Dunkerque,51.048,2.367,FRA,europe,France,FRA,5.050483,2983590.0,FRA,63125894.0


In [63]:
# clean up data (remove country code)

sealevel_context = sealevel_context.drop('ISO3v10', 1)
#sealevel_context = sealevel_context.drop('country_name', 1)
sealevel_context.head()

,id,location,latitude,longitude,country,continent,country_name,country_code,co2_emissions,coastal_population2010_sum,total_population2010_sum
0,1,Brest,48.383,-4.495,FRA,europe,France,FRA,5.050483,2983590.0,63125894.0
1,61,Marseille,43.279,5.354,FRA,europe,France,FRA,5.050483,2983590.0,63125894.0
2,453,Le Havre,49.482,0.106,FRA,europe,France,FRA,5.050483,2983590.0,63125894.0
3,467,Cherbourg,49.651,-1.636,FRA,europe,France,FRA,5.050483,2983590.0,63125894.0
4,468,Dunkerque,51.048,2.367,FRA,europe,France,FRA,5.050483,2983590.0,63125894.0


In [64]:
#Add trend data for 1985 to 2015
sealevel_context = sealevel_context.merge(trend_1985_2015, left_on='id', right_on='_id', how='left')

In [65]:
# clean up data (remove _id)
sealevel_context = sealevel_context.drop('_id', 1)

In [66]:
#Add longest trend
sealevel_context = sealevel_context.merge(trend_longest, left_on='id', right_on='_id', how='left')

In [67]:
sealevel_context.head()

,id,location,latitude,longitude,country,continent,country_name,country_code,co2_emissions,coastal_population2010_sum,total_population2010_sum,trend_1985_2015,start_year,trend_longest,_id
0,1,Brest,48.383,-4.495,FRA,europe,France,FRA,5.050483,2983590.0,63125894.0,2.385,1807.0,0.97,1.0
1,61,Marseille,43.279,5.354,FRA,europe,France,FRA,5.050483,2983590.0,63125894.0,3.530,1885.0,1.32,61.0
2,453,Le Havre,49.482,0.106,FRA,europe,France,FRA,5.050483,2983590.0,63125894.0,1.984,1959.0,2.66,453.0
3,467,Cherbourg,49.651,-1.636,FRA,europe,France,FRA,5.050483,2983590.0,63125894.0,1.908,1975.0,1.47,467.0
4,468,Dunkerque,51.048,2.367,FRA,europe,France,FRA,5.050483,2983590.0,63125894.0,NaN,1957.0,1.96,468.0


In [68]:
#Add riskindex
#sealevel_context = sealevel_context.merge(riskindex, left_on='country_name', right_on='Country', how='left')

#sealevel_context.head()

In [69]:
#sealevel_context = sealevel_context.drop('Country', 1)
#sealevel_context = sealevel_context.drop('country_name', 1)
sealevel_context = sealevel_context.drop('country', 1)
sealevel_context = sealevel_context.drop('_id', 1)

In [70]:
sealevel_context.head()

,id,location,latitude,longitude,continent,country_name,country_code,co2_emissions,coastal_population2010_sum,total_population2010_sum,trend_1985_2015,start_year,trend_longest
0,1,Brest,48.383,-4.495,europe,France,FRA,5.050483,2983590.0,63125894.0,2.385,1807.0,0.97
1,61,Marseille,43.279,5.354,europe,France,FRA,5.050483,2983590.0,63125894.0,3.530,1885.0,1.32
2,453,Le Havre,49.482,0.106,europe,France,FRA,5.050483,2983590.0,63125894.0,1.984,1959.0,2.66
3,467,Cherbourg,49.651,-1.636,europe,France,FRA,5.050483,2983590.0,63125894.0,1.908,1975.0,1.47
4,468,Dunkerque,51.048,2.367,europe,France,FRA,5.050483,2983590.0,63125894.0,NaN,1957.0,1.96


In [71]:
#Add tide data

sealevel_context = sealevel_context.merge(tide_data, left_on='id', right_on='id', how='left')

In [72]:
sealevel_context['co2_emissions'] = sealevel_context['co2_emissions'].round(decimals=2)
sealevel_context['trend_1985_2015'] = sealevel_context['trend_1985_2015'].round(decimals=2)

sealevel_context['trend_longest'] = sealevel_context['trend_1985_2015'].round(decimals=2)
sealevel_context['start_year'] = sealevel_context['start_year'].astype(object)
sealevel_context['total_population2010_sum'] = sealevel_context['total_population2010_sum'].astype(object)
sealevel_context['coastal_population2010_sum'] = sealevel_context['coastal_population2010_sum'].astype(object)
#sealevel_context['risk_rank'] = sealevel_context['risk_rank'].astype(object)
#sealevel_context['c02_rank'] = sealevel_context['c02_rank'].astype(object)

In [73]:
sealevel_context.head()

,id,location,latitude,longitude,continent,country_name,country_code,co2_emissions,coastal_population2010_sum,total_population2010_sum,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,1,Brest,48.383,-4.495,europe,France,FRA,5.05,2.98359e+06,6.31259e+07,...,133.0,135.0,140.0,157.0,158.0,162.0,164.0,176.0,169.0,NaN
1,61,Marseille,43.279,5.354,europe,France,FRA,5.05,2.98359e+06,6.31259e+07,...,144.0,147.0,161.0,204.0,200.0,210.0,217.0,226.0,191.0,NaN
2,453,Le Havre,49.482,0.106,europe,France,FRA,5.05,2.98359e+06,6.31259e+07,...,88.0,88.0,89.0,95.0,95.0,95.0,96.0,107.0,111.0,NaN
3,467,Cherbourg,49.651,-1.636,europe,France,FRA,5.05,2.98359e+06,6.31259e+07,...,3.0,6.0,10.0,21.0,21.0,23.0,22.0,30.0,22.0,NaN
4,468,Dunkerque,51.048,2.367,europe,France,FRA,5.05,2.98359e+06,6.31259e+07,...,145.0,150.0,148.0,149.0,146.0,141.0,140.0,147.0,152.0,NaN


In [74]:
sealevel_context.describe()

,id,latitude,longitude,co2_emissions,trend_1985_2015,trend_longest,1807,1808,1809,1810,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
count,513.000000,513.000000,513.000000,500.000000,424.000000,424.000000,1.0,1.0,1.0,1.0,...,509.000000,509.000000,509.000000,509.000000,504.000000,495.000000,464.000000,440.000000,418.000000,98.000000
mean,825.904483,33.845446,25.302012,10.213260,2.044340,2.044340,0.0,-51.0,-39.0,-35.0,...,50.269155,52.416503,52.640472,55.538310,59.605159,62.967677,63.368534,64.870455,66.569378,125.102041
std,546.822134,27.489068,97.354383,4.530425,3.324125,3.324125,NaN,NaN,NaN,NaN,...,194.071807,194.775793,199.683782,205.659418,207.373015,213.589543,224.117930,230.468769,230.661412,256.457769
min,1.000000,-65.246000,-177.360000,0.210000,-20.450000,-20.450000,0.0,-51.0,-39.0,-35.0,...,-1113.000000,-1129.000000,-1151.000000,-1169.000000,-1193.000000,-1211.000000,-1248.000000,-1262.000000,-1280.000000,-1293.000000
25%,368.000000,29.135000,-68.517000,6.780000,0.967500,0.967500,0.0,-51.0,-39.0,-35.0,...,-6.000000,-8.000000,-8.000000,-7.000000,1.000000,6.000000,9.000000,10.750000,2.250000,89.750000
50%,808.000000,38.317000,16.548000,9.760000,2.625000,2.625000,0.0,-51.0,-39.0,-35.0,...,66.000000,66.000000,70.000000,78.000000,77.500000,81.000000,86.000000,92.000000,96.500000,162.000000
75%,1253.000000,52.167000,129.292000,13.530000,3.890000,3.890000,0.0,-51.0,-39.0,-35.0,...,135.000000,143.000000,144.000000,157.000000,158.250000,162.000000,168.500000,175.250000,179.750000,233.000000
max,2316.000000,79.550000,178.423000,16.390000,9.440000,9.440000,0.0,-51.0,-39.0,-35.0,...,1070.000000,1053.000000,1042.000000,1060.000000,1069.000000,1079.000000,1112.000000,1127.000000,1110.000000,611.000000


In [75]:
sealevel_context.to_csv('context_data/sealevel_data_download.csv', index=False)

In [32]:
sealevel_context.to_csv('context_data/sealevel_context_data.csv', index=False)

In [33]:
sealevel_context.to_csv('data/dataviz/sealevel_context_data.csv', index=False)